In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
data_import_path = "/nfs/turbo/hrg/somar/LLM_bib/raw_data"
data_export_path = "/nfs/turbo/hrg/somar/LLM_bib/result_data"
result_path = "/nfs/turbo/hrg/somar/LLM_bib/result_file"

In [6]:
# # read the file into a pandas DataFrame
# wos_df = pd.read_csv(data_import_path+"/download_chatgpt_1001_1500.txt", delimiter='\t', header=0, index_col=False)

# # print the first 5 rows of the DataFrame
# print(wos_df.head())

  FN Clarivate Analytics Web of Science
0                                VR 1.0
1                                  PT J
2                              AU He, T
3                                 Fu, W
4                                Xu, JQ


In [3]:
wos_df = pd.DataFrame()
for file_name in sorted(os.listdir(data_import_path+"/excel_version")):
    print(file_name)
    if file_name.endswith(".xls"):
        wos_df = wos_df.append(pd.read_excel(data_import_path+"/excel_version/"+file_name))

savedrecs (2).xls
savedrecs (3).xls
savedrecs (4).xls
savedrecs (5).xls
savedrecs (6).xls
savedrecs (7).xls


In [4]:
wos_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5752 entries, 0 to 751
Data columns (total 72 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Publication Type            5752 non-null   object 
 1   Authors                     5752 non-null   object 
 2   Book Authors                10 non-null     object 
 3   Book Editors                1020 non-null   object 
 4   Book Group Authors          2209 non-null   object 
 5   Author Full Names           5752 non-null   object 
 6   Book Author Full Names      10 non-null     object 
 7   Group Authors               3 non-null      object 
 8   Article Title               5752 non-null   object 
 9   Source Title                5752 non-null   object 
 10  Book Series Title           1412 non-null   object 
 11  Book Series Subtitle        0 non-null      float64
 12  Language                    0 non-null      float64
 13  Document Type               5752 n

In [5]:
wos_df = wos_df.reset_index(drop=True)

In [6]:
wos_abstract_df = wos_df[['Article Title','DOI','Abstract']].reset_index(drop=True)

In [7]:
wos_abstract_df["Article_number"] = list(range(5752))

In [8]:
# We use title and abstract, as well focus on sentence-line context aware info, not keywords.
wos_abstract_df["Title_abstract"] = wos_abstract_df["Article Title"]+". "+wos_abstract_df["Abstract"]

In [9]:
# add dates
all_years = []
for i in range(len(wos_df)):
    if not np.isnan(wos_df["Publication Year"][i]):
        all_years.append(int(wos_df["Publication Year"][i]))
    elif type(wos_df["Early Access Date"][i])==str:
        all_years.append(int(wos_df["Early Access Date"][i][-4:]))
    else:
        all_years.append("")

In [10]:
all_years.count("")

0

In [11]:
wos_abstract_df["Year"] = all_years

### Topic Modeling

In [12]:
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
from umap import UMAP
from sklearn.feature_extraction.text import CountVectorizer
from hdbscan import HDBSCAN
from sklearn.cluster import KMeans

In [13]:
docs = [str(item) for item in list(wos_abstract_df.Title_abstract)]

In [14]:
vectorizer_model = CountVectorizer(stop_words="english")
# hdbscan_model = HDBSCAN(min_cluster_size=10,
#                         min_samples=5,
#                         metric='euclidean',
#                         cluster_selection_method='eom',
#                         prediction_data=True) # random_state=42
cluster_model = KMeans(n_clusters=200)
topic_model = BERTopic(hdbscan_model=cluster_model, #hdbscan_model,
                       vectorizer_model=vectorizer_model,
                       top_n_words=10) # nr_topics=100,
topics, probs = topic_model.fit_transform(docs)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [15]:
topic_model.save(result_path+"/LLM_wos_abstract_200")

/home/lizhouf/.local/lib/python3.9/site-packages/scipy/sparse/_index.py:146: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


In [16]:
wos_abstract_df["topics"] = topics

In [17]:
wos_abstract_df

,Article Title,DOI,Abstract,Article_number,Title_abstract,Year,topics
0,Jigsaw: Large Language Models meet Program Syn...,10.1145/3510003.3510203,Large pre-trained language models such as GPT-...,0,Jigsaw: Large Language Models meet Program Syn...,2022,67
1,Structured Pruning of Large Language Models,NaN,Large language models have recently achieved s...,1,Structured Pruning of Large Language Models. L...,2020,16
2,Playing Games with Ais: The Limits of GPT-3 an...,10.1007/s11023-022-09602-0,This article contributes to the debate around ...,2,Playing Games with Ais: The Limits of GPT-3 an...,2022,144
3,Extracting Training Data from Large Language M...,NaN,It has become common to publish large (billion...,3,Extracting Training Data from Large Language M...,2021,125
4,Exploring Large Language Models in a Limited R...,10.1109/Confluence51648.2021.9377081,Generative Pre-trained Transformers (GPT) have...,4,Exploring Large Language Models in a Limited R...,2021,85
...,...,...,...,...,...,...,...
5747,Global User-Level Perception of COVID-19 Conta...,10.2196/36238,Background: Contact tracing has been globally ...,5747,Global User-Level Perception of COVID-19 Conta...,2022,24
5748,Pretrained Transformer Language Models Versus ...,10.2196/34834,"Background: In recent years, social media has ...",5748,Pretrained Transformer Language Models Versus ...,2022,55
5749,Robust effect of justice for others: Evidence ...,10.1360/TB-2019-0694,Maintenance of control and non-randomness is t...,5749,Robust effect of justice for others: Evidence ...,2020,42
5750,TOPAS/Geant4 configuration for ionization cham...,10.1088/1361-6560/aac30e,Monte Carlo (MC) calculations are a fundamenta...,5750,TOPAS/Geant4 configuration for ionization cham...,2018,70


In [18]:
# show topics info
topic_model.get_topic_info()

,Topic,Count,Name
0,0,98,0_aspect_sentiment_aspectbased_absa
1,1,98,1_ranking_retrieval_query_document
2,2,89,2_visual_image_vqa_captioning
3,3,82,3_protein_proteins_molecular_dna
4,4,80,4_hate_speech_offensive_hateful
...,...,...,...
195,195,5,195_covid19_pandemic_relationships_spatial
196,196,5,196_kerasbert_equivalence_judging_processbert
197,197,4,197_music_sheet_musical_instrument
198,198,4,198_coherence_utterancepair_ffcd_unilm


In [19]:
topic_model.get_topic_info().to_csv(data_export_path+"/LLM_wos_abstract_topics_metadata_200.csv",index=False)

In [20]:
# show Topic Positions -> suggestions for further clustering
#topic_model.visualize_topics()
fig_IDM = topic_model.visualize_topics()
fig_IDM.write_html(result_path+"/LLM_wos_abstract_viz_IDM_200.html")

In [21]:
# visualize Topic Hierarchy -> suggestions for further clustering
fig_hierarchy = topic_model.visualize_hierarchy()
fig_hierarchy.write_html(result_path+"/LLM_wos_abstract_viz_hierarchy_200.html")

In [22]:
# visualize words
fig_topic_word_scores = topic_model.visualize_barchart(top_n_topics=32)
fig_topic_word_scores.write_html(result_path+"/LLM_wos_abstract_viz_topic_word_scores_200.html")

In [23]:
# visualize topic similarity -- not useful in our case, maybe include to show not correlated
fig_heatmap = topic_model.visualize_heatmap()
fig_heatmap.write_html(result_path+"/LLM_wos_abstract_viz_heatmap_200.html")

In [24]:
# visualize documents
sentence_model = SentenceTransformer("all-MiniLM-L6-v2")
embeddings = sentence_model.encode(docs, show_progress_bar=False)
# Run the visualization with the original embeddings
topic_model.visualize_documents(docs, embeddings=embeddings)
# Reduce dimensionality of embeddings, this step is optional but much faster to perform iteratively:
reduced_embeddings = UMAP(n_neighbors=10, n_components=2, min_dist=0.0, metric='cosine').fit_transform(embeddings)
# save embeddings (for Tableau)
wos_abstract_df["x"] = reduced_embeddings[:, 0]
wos_abstract_df["y"] = reduced_embeddings[:, 1]


In [25]:
import pickle
with open(result_path+'/LLM_wos_abstract_embeddings_200.pickle', 'wb') as pkl:
    pickle.dump(embeddings, pkl)

In [26]:
wos_abstract_df.to_csv(data_export_path+"/LLM_wos_abstract_w_topics_years_positions_200.csv",index=False)

In [27]:
### Topic over time
topics_over_time = topic_model.topics_over_time(docs, all_years)

In [124]:
# will export tableau viz after finish
# topics_over_time_eg_viz = topic_model.visualize_topics_over_time(topics_over_time, topics=[13, 2, 8, 10, 40, 47])
# topics_over_time_eg_viz.write_html(result_path+"/LLM_wos_abstract_viz_topics_over_time_eg.html")

TODO:
1. Remove nan docs
2. Group the 50 topics, or choose interesting ones

Explore fields
1. get unique
2. coref
3. detect ROR
4. analyze collaboration

Refs:
- https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5065891/
- https://core.ac.uk/download/pdf/25068797.pdf

Note: we don't care about co-citation, we just aggregate collaboration in single publications

In [28]:
wos_df.Affiliations

0                          Microsoft; Stanford University
1                                                     NaN
2          University of Warsaw; University of Nottingham
3       Google Incorporated; Stanford University; Univ...
4       Indian Institute of Technology System (IIT Sys...
                              ...                        
5747    Qatar Foundation (QF); Hamad Bin Khalifa Unive...
5748    University of Limerick; Taibah University; Uni...
5749    Xiamen University; Xiamen University; Xiamen U...
5750    University Hospital of Giessen & Marburg; Univ...
5751                                 Princeton University
Name: Affiliations, Length: 5752, dtype: object

In [32]:
# wos_df.Addresses[2]

'[Sobieszek, Adam] Univ Warsaw, Dept Psychol, Warsaw, Poland; [Price, Tadeusz] Univ Nottingham, Dept Philosophy, Nottingham, England'